# Regressão - Preço de Venda da Casa

Este notebook realiza um estudo, um conjunto de experimentos, de algoritmos de regressão sobre o dataset [House Sales in King County, USA](https://www.kaggle.com/harlfoxem/housesalesprediction). Um conjunto de dados que reúne mais de 21 mil casas e 21 atributos, tais como preço, número de quartos, número de banheiros, andares, nota da casa, entre outros. Nosso objetivo é predizer o valor de uma casa baseado nas características da casa.

> Conteúdo voltado para iniciantes na área de Aprendizado de Máquina e Ciência de Dados!

<a id="top"></a>

## Conteúdo

> **Nota**. Alguns códigos foram ocultados a fim de facilitar a leitura e dar destaque para os conteúdos mais importantes.

O notebook está organizado como segue:

- [Dados](#data) - Carregamento dos dados, pré-processamento.
- [Visualização](#visual) - Análise exploratória dos dados.
- [Regressão](#regression) - Aplicação de algoritmos de Aprendizado de Máquina.
    - [KNN Regressor](#knn) - Regressão com k-NN.
    - [Regressão Linear](#reg) - Regressão com Regressão Linear.
    - [Support Vector Machines](#svm) - Regressão com Support Vector Machines.
    - [Árvore de Decisão](#decision) - Regressão com Decision Tree.
    - [Random Forest](#forest) - Regressão com Random Forest.
    - [Bagging](#bagging) - Regressão com estratégia de Bagging.
    - [Ensemble](#ensemble) - Regressão com estratégia de Ensemble.
    - [AutoML](#automl) - Regressão com Automated Machine Learning.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install auto-sklearn==0.12.0
!pip install scikit-learn==0.23.2

<a id="data"></a>

-----

# Dados

Esta seção reúne um conjunto de código para carregamento e pré-processamento sobre os dados.

[Voltar para o Topo](#top)


## Carregamento dos Dados

In [ ]:
# processamento de dados, algebra linear
import numpy as np 
import pandas as pd

In [ ]:
# imprime os arquivos
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/housesalesprediction/kc_house_data.csv')
df.sample(3)

## Seleção dos Dados

Nesta seção observamos os dados e selecionamos apenas aqueles que são interessantes para os modelos de regressão. Além disso, não será proposto nenhum _feature engineer_ para enriquecimento dos dados ou tratamendo dos dados.

**Descrição dos Dados**

[Column defintions - Nova19](https://www.kaggle.com/harlfoxem/housesalesprediction/discussion/207885)

- id - ID unico para cada casa. _(remover)_
- date - Data da casa a venda. _(remover)_
- price - Preço da cada.
- bedrooms - Número de quartos.
- bathrooms - Número de banheiros, no qual .5 conta como lavabo.
- sqft_living - M2 do espaço interior.
- sqft_lot - M2 do espaço do terreno.
- floors - Número de andares.
- waterfront - Tem vista para o mar (1) ou não (0). (categórico)
- view - Valor de 0 a 4 informando se a vista é boa. (categórico)
- condition - Valor de 1 a 5 sobre a condição da casa. (categórico)
- grade - Nota de 1 a 13, no qual 1-3 pequenas construções, 7 construção e desing mediano, e 11-13 para construções de alto nível.
- sqft_above - M2 do interior da casa, acima do nível do solo.
- sqft_basement - M2 do interior da casa, abaixo do nível do solo.
- yr_built - Ano de construção da casa.
- yr_renovated - Último ano de renovação da casa. _(remover)_
- zipcode - CEP da residência. _(remover)_
- lat - Latitude.
- long - Longitude.
- sqft_living15 - M2 do espaço interno para os 15 vizinhos mais próximos.
- sqft_lot15 - M2 do terreno para os 15 vizinhos mais próximos.

In [ ]:
columns_to_remove = ['id', 'date', 'yr_renovated', 'zipcode']
df = df.drop(columns_to_remove, axis=1)
df.sample(3)

Visualizando a estatística descritiva dos imóveis a venda.

In [ ]:
df.describe()

In [ ]:
lines, columns = df.shape
print('linhas :', lines)
print('colunas:', columns)

## Conjunto de Treinamento e Teste

Nesta seção vamos separa os valores de `X` e `Y`, em seguida normalizar os valores de `X`, por fim, separar entre conjunto de dados de treinamento e teste.

> A normalização se faz necessária, pois alguns algoritmos se beneficiam de valores normalizados, tal como o K-NN.

In [ ]:
# recupera os valores (X), e as classes (Y)
X = df.drop('price', axis=1)
Y = df['price']

### Normalização dos Dados

Nesta seção vamos utilizar a normalização [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Esta função de preprocessamento normaliza os dados conforme segue: 

$$x_{new} = \frac{(x - \overline{x})}{\sigma}$$

Ou seja, o novo valor $x_{new}$ é resultado da normalização do $x$, utilizando a média $\overline{x}$ e o desvio padrão $\sigma$.

In [ ]:
# normalizador
from sklearn.preprocessing import StandardScaler

In [ ]:
# normalização dos dados
min_max_scaler = StandardScaler()
X = min_max_scaler.fit_transform(X)

### Conjuntos de Dados

In [ ]:
# treinamento, test split
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=26)

In [ ]:
print('treinamento:', len(y_train))
print('teste      :', len(y_test))

<a id="visual"></a>

-----

# Visualização dos Dados

Esta seção reúne um conjunto de visualizações sobre os dados.

[Voltar para o Topo](#top)


In [ ]:
# visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

### Qual a correlação dos atributos?

`DataFrame.corr()` calcula a correlação de pares de colunas, excluindo `NaN` e valores nulos. Por padrão é computado a [Correlação de Pearson](https://www.statisticssolutions.com/pearsons-correlation-coefficient/), seu coeficiente de correlação mede a relação estatística, ou associação, entre duas variáveis contínuas. 

In [ ]:
# extraí a correlação dos dados
corr = df.corr(method='pearson')

# heatmap - gráfico de calor
plt.figure(figsize=(11,8))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()
# print(corr)

### Histograma dos Valores por Atributo

In [ ]:
df.hist(figsize=(10,8))
plt.tight_layout()
plt.show()

<a id="regression"></a>

-----

# Regressão

Esta seção reúne um conjunto de experimentos. Cada subseção é um algoritmo de Aprendizado de Máquina.


[Voltar para o Topo](#top)

In [ ]:
# métricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# variável de resultado final
# será armazenado o resultado de todos experimentos
experiment = {}

<a id="knn"></a>

## K-NN Regressor

_(k-Nearest Neighbors)_

In [ ]:
# regressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
model1 = KNeighborsRegressor(n_neighbors=3,metric='euclidean')
model1.fit(X_train,y_train)

### Avaliação

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
# R Square Error
r2 = r2_score(y_test,y_pred)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test,y_pred)

# Mean Square Error (MSE)
mse = mean_squared_error(y_test,y_pred, squared=True)

# Root Mean Square Error (RMSE)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['KNN'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão KNN Regressor**   

k-NN obteve um R² próximo de 79%, ou seja, representou bem a função de preços.

-----

<a id="reg"></a>

## Regressão Linear

In [ ]:
# regressor
from sklearn.linear_model import LinearRegression

In [ ]:
model2 = LinearRegression()
model2.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Linear Regression'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão Regressão Linear**   

Regressão Linear obteve um resultado inferior ao k-NN, com R² de 70%.

-----

<a id="svm"></a>

## Support Vector Machines (SVM)

In [ ]:
# regressor
from sklearn.svm import SVR

In [ ]:
model3 = SVR()
model3.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model3.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['SVM'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão Support Vector Machines (SVM)**   

SVM não conseguiu nem atinguir o caso médio, pois seu R² está negativo.

-----

<a id="decision"></a>

## Árvore de Decisão

In [ ]:
# regressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model4 = DecisionTreeRegressor(random_state=26)
model4.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model4.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Decision Tree'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

### Visualização

Nós conseguimos visualizar a árvore de decisão, como as ramificações ocorreram.   
É muito útil para uma apresentação de negócio, em que você consegue explicar a inteligência induzida.

> Não será apresentado neste notebook, pois a árvore aqui construída é muito grande e demora para ser executada.

- Referência: [Visualize a Decision Tree in 4 Ways with Scikit-Learn and Python](https://mljar.com/blog/visualize-decision-tree/)

**Discussão Árvore de Decisão**   

Árvore de Decisão obteve um resultado similar à Regressão Linear, com R² de 71%.

-----

<a id="forest"></a>

## Random Forest

In [ ]:
# regressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model5 = RandomForestRegressor(n_estimators=100, random_state=26)
model5.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model5.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Random Forest'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão Random Forest**   

Random Forest obteve o melhor resultado até o momento, R² de 88%.   

Além disso, Random Forests são um dos algoritmos mais utilizados em competições de Aprendizado de Máquina.

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="bagging"></a>

## Bagging

Regressão com estratégia de Bagging, com algoritmo base Decision Tree.

In [ ]:
# ensemble
from sklearn.ensemble import BaggingRegressor

In [ ]:
model_base = DecisionTreeRegressor(random_state=26)
model6 = BaggingRegressor(base_estimator=model_base, n_estimators=10, random_state=26)
model6.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model6.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Bagging'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão Bagging**   

Bagging obteve bons resultados, próximo ao Random Forest.   

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="ensemble"></a>

## Ensemble

Regressão com estratégia de Ensemble, utilizando os algoritmos Linear Regression e Random Forest.

In [ ]:
# ensemble
from sklearn.ensemble import VotingRegressor

In [ ]:
r1 = LinearRegression()
r2 = RandomForestRegressor(n_estimators=10, random_state=26)

model7 = VotingRegressor([('LR', r1), ('RF', r2)])
model7.fit(X_train, y_train)

### Avaliação

In [ ]:
y_pred = model7.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['Ensemble'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

**Discussão Ensemble**   

Ensemble obteve bons resultados, próximo ao Bagging.   

> **Nota**. Possui alto custo computacional, pois tem que treinar vários modelos.

-----

<a id="automl"></a>

## AutoML

Automated Machine Learning.

In [ ]:
# automl
import autosklearn.regression

In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/automl/tmp/',
    output_folder='/automl/output/',
)
automl.fit(X_train, y_train, dataset_name='housesalesprediction')

### Avaliação

In [ ]:
y_pred = automl.predict(X_test)

In [ ]:
r2 = r2_score(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred, squared=True)
rmse = mean_squared_error(y_test,y_pred, squared=False)

In [ ]:
experiment['AutoML'] = {'R2':r2, 'MAE':mae, 'MSE':mse, 'RMSE':rmse}

print('R2  :',r2)
print('MAE :',mae)
print('MSE :',mse)
print('RMSE:',rmse)

### Visualização

Podemos ver o modelo ou o ensemble de modelos utilizado no AutoML.

> Para isto, utilize o comando `automl.show_models()`.

In [ ]:
automl.show_models()

# Conclusão

Por fim, o melhor algoritmo foi o Random Forest com R² de 88%.   
As demais estratégias de ensemble, Bagging e Ensemble, também apresentaram bons resultados.

In [ ]:
# palheta de cores
import seaborn as sns

In [ ]:
cm = sns.color_palette('Blues_r', as_cmap=True)
pd.DataFrame(experiment).T.style.background_gradient(subset=['R2'], cmap=cm).highlight_max(subset=['R2'], axis=0)